In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.models as models
from torch.utils.data import DataLoader

import imageio

import random
import os
import copy

import torch

import sys
sys.path.append("../../")

from src.model.TransferModel import get_style_model_and_losses
from src.dataset.dataset import ImgDataset
from src.utils.other import *
from src.utils.plotting import canvas2rgb_array

from torchvision import transforms

# grid of annie, kennen, mf

In [46]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
imsize = 256 if torch.cuda.is_available() else 128  # use small size if no gpu

style_dataset = ImgDataset("../../data/traditional_style_images/", imsize, device)
style_loader = DataLoader(style_dataset, batch_size=1, shuffle=False)

outfolder = "../../results/evolution/traditional_style_images_256/"
os.system("mkdir -p " + str(outfolder))

# the rest of this cell from
# https://pytorch.org/tutorials/advanced/neural_style_tutorial.html
cnn = models.vgg19(pretrained=True).features.to(device).eval()

cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

# desired depth layers to compute style/content losses :
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

0

In [47]:
# https://pytorch.org/tutorials/advanced/neural_style_tutorial.html
def imshow(tensor, ax, title=None, fontsize=None):
    unloader = transforms.ToPILImage()  # reconvert into PIL image
    image = tensor.cpu().clone() # we clone the tensor to not do changes on it
    image = image.squeeze()      # remove the fake batch dimension
    image = unloader(image)
    ax.imshow(image)
    if title is not None:
        if fontsize is None:
            ax.set_title(title)
        else:
            ax.set_title(title, fontsize=fontsize)
    #ax.pause(0.001) # pause a bit so that plots are updated

In [49]:
# https://pytorch.org/tutorials/advanced/neural_style_tutorial.html
# we modified this method to allow us to save the image during optimization
def run_style_transfer(cnn, normalization_mean, normalization_std,
                       content_img, style_img, input_img, num_steps=300,
                       style_weight=1000000, content_weight=1, 
                       content_layers=['conv_4'],
                       style_layers=['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5'],
                       do_print=True, device="cpu", save_interval=10, fpath=None):
    """Run the style transfer."""
    if do_print:
        print('Building the style transfer model..')
    model, style_losses, content_losses = get_style_model_and_losses(cnn,
        normalization_mean, normalization_std, style_img, content_img, device,
        content_layers, style_layers)

    # We want to optimize the input and not the model parameters so we
    # update all the requires_grad fields accordingly
    input_img.requires_grad_(True)
    model.requires_grad_(False)

    optimizer = optim.LBFGS([input_img])
    
    unloader = transforms.ToPILImage()  # reconvert into PIL image
    
    if do_print:
        print('Optimizing..')
    run = [0]
    count = 0
    images = []
    fig = plt.figure(figsize=(3, 3))
    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    image = input_img.clone().clamp_(0, 1).cpu()
    image = image.squeeze()
    image = unloader(image)
    plt.imshow(image)
    plt.close()
    data = canvas2rgb_array(fig.canvas)
    images.append(data)

    while run[0] <= num_steps:

        def closure():
            # correct the values of updated input image
            with torch.no_grad():
                input_img.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            style_score = 0
            content_score = 0

            for sl in style_losses:
                style_score += sl.loss
            for cl in content_losses:
                content_score += cl.loss

            style_score *= style_weight
            content_score *= content_weight

            loss = style_score + content_score
            loss.backward()

            run[0] += 1
            if run[0] % 50 == 0:
                if do_print:
                    print("run {}:".format(run))
                    print('Style Loss : {:4f} Content Loss: {:4f}'.format(
                        style_score.item(), content_score.item()))
                    print()

            return style_score + content_score

        optimizer.step(closure)
        count += 10
        
        if count % save_interval == 0:
            #fig, ax = plt.subplots(1, 1, figsize=(5, 5))
            #fig = plt.figure(figsize=(5, 5))
            fig = plt.figure(figsize=(3, 3))
            ax = plt.gca()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            image = input_img.clone().clamp_(0, 1).cpu()
            image = image.squeeze()
            image = unloader(image)
            plt.imshow(image)
            plt.close()
            data = canvas2rgb_array(fig.canvas)
            images.append(data)
    imageio.mimsave(fpath + ".gif", images, duration=0.33)

In [50]:
steps=300
style_weight=1000000
content_weight=1
fontsize= 15
interval=10

# get annie data
annie_content = style_dataset.load_image("../../data/Annie_2-2.jpg", device)
for i, annie_style in enumerate(style_loader):
    if i == 5:
        break      
annie_style = annie_style.squeeze(0)

# annie
run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                  annie_content, annie_style, annie_content.clone(), num_steps=steps,
                                  style_weight=style_weight, content_weight=content_weight,
                                  do_print=True, device=device, save_interval=interval,
                                  fpath=outfolder + "annie")

Building the style transfer model..
Optimizing..
run [50]:
Style Loss : 310.367218 Content Loss: 20.954500

run [100]:
Style Loss : 90.559570 Content Loss: 24.542795

run [150]:
Style Loss : 41.321701 Content Loss: 25.874041

run [200]:
Style Loss : 25.920315 Content Loss: 25.775944

run [250]:
Style Loss : 18.034121 Content Loss: 25.633121

run [300]:
Style Loss : 13.496673 Content Loss: 25.218895



In [51]:
steps=300
style_weight=1000000
content_weight=1
fontsize= 15

interval=10

# get kennen data
kennen_content = style_dataset.load_image("../../data/Kennen_3.jpg", device)
for i, kennen_style in enumerate(style_loader):
    if i == 6:
        break
kennen_style = kennen_style.squeeze(0)


# kennen
run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                   kennen_content, kennen_style, kennen_content.clone(), num_steps=steps,
                                   style_weight=style_weight, content_weight=content_weight,
                                   do_print=True, device=device, save_interval=interval,
                                   fpath=outfolder + "kennen")

Building the style transfer model..
Optimizing..
run [50]:
Style Loss : 1896.307617 Content Loss: 48.923653

run [100]:
Style Loss : 379.989746 Content Loss: 53.387779

run [150]:
Style Loss : 161.407211 Content Loss: 53.977608

run [200]:
Style Loss : 84.243683 Content Loss: 54.779457

run [250]:
Style Loss : 48.499767 Content Loss: 55.218140

run [300]:
Style Loss : 31.059223 Content Loss: 55.247589



In [52]:
steps=300
style_weight=1000000
content_weight=1
fontsize= 15

interval=10

# get mf data
mf_path = "../../data/missfortuneloadscreen_17.skins_missfortune_skin15_chromas-300x545.jpg"
mf_content = style_dataset.load_image(mf_path, device)
for i, mf_style in enumerate(style_loader):
    pass
mf_style = mf_style.squeeze(0)

# mf
run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                               mf_content, mf_style, mf_content.clone(), num_steps=steps,
                               style_weight=style_weight, content_weight=content_weight,
                               do_print=True, device=device, save_interval=interval,
                               fpath=outfolder + "mf")


Building the style transfer model..
Optimizing..
run [50]:
Style Loss : 161.376236 Content Loss: 20.096439

run [100]:
Style Loss : 53.316444 Content Loss: 19.499821

run [150]:
Style Loss : 33.284462 Content Loss: 18.431530

run [200]:
Style Loss : 24.201576 Content Loss: 17.339361

run [250]:
Style Loss : 18.293856 Content Loss: 16.483480

run [300]:
Style Loss : 13.727413 Content Loss: 15.889206



In [53]:
steps=300
style_weight=1000000
content_weight=1
fontsize= 15

interval=10

# get mf data
jhin_path = "../../data/jhin-bloodmoon-300x545.jpg"
jhin_content = style_dataset.load_image(jhin_path, device)
for i, jhin_style in enumerate(style_loader):
    if i == 1:
        break
jhin_style = jhin_style.squeeze(0)

# mf
run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                               jhin_content, jhin_style, jhin_content.clone(), num_steps=steps,
                               style_weight=style_weight, content_weight=content_weight,
                               do_print=True, device=device, save_interval=interval,
                               fpath=outfolder + "jhin")


Building the style transfer model..
Optimizing..
run [50]:
Style Loss : 288.739349 Content Loss: 40.792854

run [100]:
Style Loss : 70.445267 Content Loss: 42.472816

run [150]:
Style Loss : 30.702358 Content Loss: 42.772461

run [200]:
Style Loss : 19.018688 Content Loss: 42.511086

run [250]:
Style Loss : 11.512776 Content Loss: 42.204685

run [300]:
Style Loss : 8.646844 Content Loss: 41.623611



In [54]:
steps=300
style_weight=1000000
content_weight=1
fontsize= 15

interval=10

# get mf data
img_path = "../../data/Garen_5-300x545.jpg"
img_content = style_dataset.load_image(img_path, device)
for i, img_style in enumerate(style_loader):
    if i == 0:
        break
img_style = img_style.squeeze(0)

# mf
run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                               img_content, img_style, img_content.clone(), num_steps=steps,
                               style_weight=style_weight, content_weight=content_weight,
                               do_print=True, device=device, save_interval=interval,
                               fpath=outfolder + "garen")


Building the style transfer model..
Optimizing..
run [50]:
Style Loss : 37.206230 Content Loss: 31.795567

run [100]:
Style Loss : 11.350952 Content Loss: 28.921564

run [150]:
Style Loss : 5.424634 Content Loss: 24.815208

run [200]:
Style Loss : 3.278734 Content Loss: 21.240761

run [250]:
Style Loss : 2.331804 Content Loss: 18.814152

run [300]:
Style Loss : 1.887944 Content Loss: 17.154497



In [55]:
steps=300
style_weight=1000000
content_weight=1
fontsize= 15

interval=10

# get mf data
img_path = "../../data/Zilean_4-300x545.jpg"
img_content = style_dataset.load_image(img_path, device)
for i, img_style in enumerate(style_loader):
    if i == 3:
        break
img_style = img_style.squeeze(0)

# mf
run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                               img_content, img_style, img_content.clone(), num_steps=steps,
                               style_weight=style_weight, content_weight=content_weight,
                               do_print=True, device=device, save_interval=interval,
                               fpath=outfolder + "zilean")


Building the style transfer model..
Optimizing..
run [50]:
Style Loss : 227.244949 Content Loss: 21.349361

run [100]:
Style Loss : 93.222656 Content Loss: 22.098766

run [150]:
Style Loss : 52.769402 Content Loss: 22.247578

run [200]:
Style Loss : 34.929718 Content Loss: 21.496502

run [250]:
Style Loss : 23.799997 Content Loss: 20.737022

run [300]:
Style Loss : 15.932960 Content Loss: 20.188984

